<a href="https://colab.research.google.com/github/ishutinavi/Six_handshakes/blob/main/script_six_handshakes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
from bs4 import BeautifulSoup
import time


def is_correct_language(url, expected_language):
    return url.startswith(f"https://{expected_language}.wikipedia.org")


def get_wikipedia_links(url, base_url, expected_language):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        links = set()
        for a in soup.select('div.mw-parser-output a[href^="/wiki/"]'):
            href = a.get('href')
            if ":" not in href:
                full_url = base_url + href
                if is_correct_language(full_url, expected_language):
                    links.add(full_url)

        return links
    except requests.exceptions.RequestException as e:
        return set()


def find_first_path(start_url, target_url, rate_limit):
    # Определяем язык на основе доменов
    start_language = start_url.split("//")[1].split(".")[0]
    target_language = target_url.split("//")[1].split(".")[0]

    if start_language != target_language:
        print("Языки начального и конечного URL не совпадают.")
        return None

    visited = set()
    stack = [(start_url, [start_url])]
    requests_made = 0

    while stack:
        current_url, path = stack.pop()

        if current_url == target_url:
            return path

        if len(path) >= 5:
            continue

        if current_url not in visited:
            visited.add(current_url)
            links = get_wikipedia_links(current_url, base_url=f"https://{start_language}.wikipedia.org", expected_language=start_language)
            requests_made += 1

            if requests_made >= rate_limit:
                time.sleep(1)
                requests_made = 0

            for link in links:
                if link not in visited:
                    stack.append((link, path + [link]))

    return None


def main():
    # Получение входных данных от пользователя
    url1 = input("Введите начальный URL (например, https://en.wikipedia.org/wiki/Example): ")
    url2 = input("Введите конечный URL (например, https://en.wikipedia.org/wiki/Another_Example): ")
    rate_limit = int(input("Введите лимит запросов на 1 секунду: "))

    print("Поиск пути от url1 к url2")
    path1 = find_first_path(url1, url2, rate_limit)
    if path1:
        print(f"Путь от {url1} к {url2}:")
        print(" -> ".join(path1))
    else:
        print(f"Путь от {url1} к {url2} не найден за 5 шагов.")

    print("Поиск пути от url2 к url1")
    path2 = find_first_path(url2, url1, rate_limit)
    if path2:
        print(f"Путь от {url2} к {url1}:")
        print(" -> ".join(path2))
    else:
        print(f"Путь от {url2} к {url1} не найден за 5 шагов.")

    if path1 and path2:
        print("Теория шести рукопожатий подтверждается для выбранных статей.")
    else:
        print("Теория шести рукопожатий не подтверждается для выбранных статей.")


if __name__ == "__main__":
    main()

Введите начальный URL (например, https://en.wikipedia.org/wiki/Example): https://en.wikipedia.org/wiki/Six_degrees_of_separation
Введите конечный URL (например, https://en.wikipedia.org/wiki/Another_Example): https://en.wikipedia.org/wiki/American_Broadcasting_Company
Введите лимит запросов на 1 секунду: 10
Поиск пути от url1 к url2
Путь от https://en.wikipedia.org/wiki/Six_degrees_of_separation к https://en.wikipedia.org/wiki/American_Broadcasting_Company:
https://en.wikipedia.org/wiki/Six_degrees_of_separation -> https://en.wikipedia.org/wiki/Small-world_experiment -> https://en.wikipedia.org/wiki/Mark_Granovetter -> https://en.wikipedia.org/wiki/Johns_Hopkins_University -> https://en.wikipedia.org/wiki/American_Broadcasting_Company
Поиск пути от url2 к url1
Путь от https://en.wikipedia.org/wiki/American_Broadcasting_Company к https://en.wikipedia.org/wiki/Six_degrees_of_separation:
https://en.wikipedia.org/wiki/American_Broadcasting_Company -> https://en.wikipedia.org/wiki/Hulu -> h